<a href="https://colab.research.google.com/github/myeongseop2/NIDS_project/blob/main/Probe/DNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dropout, Dense
from keras.callbacks import ModelCheckpoint, CSVLogger

MODEL_SAVE_FOLDER_PATH = '/content/drive/MyDrive/project/NIDS/model'
MODEL_HISTORY_FOLDER_PATH = '/content/drive/MyDrive/project/NIDS/history'

if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
if not os.path.exists(MODEL_HISTORY_FOLDER_PATH):
    os.mkdir(MODEL_HISTORY_FOLDER_PATH)

x_train = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/x_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/y_train.csv')
x_test = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/x_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/y_test.csv')

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train_rnn = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test_rnn = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(19,)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model_path = MODEL_SAVE_FOLDER_PATH + 'DNN_{epoch}.h5'
history_path = MODEL_HISTORY_FOLDER_PATH + 'log.csv'

csv_logger = CSVLogger(filename=history_path, append=True, separator=';')
check_point = ModelCheckpoint(filepath=model_path, verbose=0, save_best_only=False,save_weights_only=False, mode='auto', save_freq='epoch', period=10000)

model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.fit(x_train_rnn, y_train, epochs=10, validation_data=(x_test_rnn, y_test), callbacks=[check_point, csv_logger])

pred = model.predict(x_test_rnn)

threshold = 0.5

pred2 = np.where(pred >= threshold, 1, pred)
pred2 = np.where(pred2 < threshold, 0, pred2)

print(pred2)

cm = confusion_matrix(y_test, pred2)

print("TN: ", cm[0][0])
print("FP: ", cm[0][1])
print("FN: ", cm[1][0])
print("TP: ", cm[1][1])

print("Detection Rate is : ", (cm[1][1])/(cm[1][1]+cm[1][0])*100)
print("False Alarm Rate is : ", (cm[0][1])/(cm[0][1]+cm[0][0])*100)

print(classification_report(y_test, pred2, digits=5))